In [ ]:
import os
import cv2
from avapi.carla import CarlaScenesManager


cpath = os.path.join("/data/shared/CARLA/multi-agent-visualization")
CSM = CarlaScenesManager(cpath)
print(CSM.scenes)
idx = 1
CDM = CSM.get_scene_dataset_by_index(idx)
print(f"{len(CDM)} frames")

save_folder = f"visualization_output_{idx}"
os.makedirs(save_folder, exist_ok=True)

In [ ]:
from avstack.modules.perception.object2dfv import MMDetObjectDetector2D
from avstack.modules.perception.object3d import MMDetObjectDetector3D
from avapi.visualize.snapshot import show_lidar_bev_with_boxes, show_lidar_on_image, show_image_with_boxes
from avapi.visualize.movie import make_movie

In [ ]:
detector3d_veh = MMDetObjectDetector3D(model="pointpillars", dataset="carla-vehicle")
detector3d_inf = MMDetObjectDetector3D(model="pointpillars", dataset="carla-infrastructure")

In [ ]:
frame_idx = 55

# BEV zoomed in
agent = 5
camera_sensor = "camera-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
im_out = show_image_with_boxes(img=img, boxes=[], inline=True, return_image=True)
im_out = cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB) 
cv2.imwrite(os.path.join(save_folder, "bev-full.png"), im_out)

# BEV from camera
agent = 4
camera_sensor = "camera-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
im_out = show_image_with_boxes(img=img, boxes=[], inline=True, return_image=True)
im_out = cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB) 
cv2.imwrite(os.path.join(save_folder, "bev-zoomed.png"), im_out)

# ego agent view
agent = 0
camera_sensor = "camera-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
im_out = show_image_with_boxes(img=img, boxes=[], inline=True, return_image=True)
im_out = cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB) 
cv2.imwrite(os.path.join(save_folder, "ego-camera.png"), im_out)



In [ ]:
# ego agent view
agent = 0
camera_sensor = "camera-0"
lidar_sensor = "lidar-0"
frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
# for some reason it filtered the next door box...
# objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
objs = detector3d_veh(pc)

im_out = show_lidar_bev_with_boxes(pc=pc, boxes=objs, return_image=True)
im_out = cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB) 
cv2.imwrite(os.path.join(save_folder, "ego-lidar-bev.png"), im_out)

im_out = show_image_with_boxes(img=img, boxes=objs, inline=True, return_image=True)
im_out = cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB) 
cv2.imwrite(os.path.join(save_folder, "ego-camera-boxes.png"), im_out)

# infrastructure views
for agent in range(1, 4):
    camera_sensor = "camera-0"
    lidar_sensor = "lidar-0"
    frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
    img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
    pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
    # objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)
    objs = detector3d_inf(pc)

    im_out = show_lidar_bev_with_boxes(pc=pc, boxes=objs, return_image=True)
    im_out = cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB) 
    cv2.imwrite(os.path.join(save_folder, f"agent-{agent}-lidar-bev.png"), im_out)

    im_out = show_image_with_boxes(img=img, boxes=[], inline=True, return_image=True)
    im_out = cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB) 
    cv2.imwrite(os.path.join(save_folder, f"agent-{agent}-camera.png"), im_out)

    im_out = show_image_with_boxes(img=img, boxes=objs, inline=True, return_image=True)
    im_out = cv2.cvtColor(im_out, cv2.COLOR_BGR2RGB) 
    cv2.imwrite(os.path.join(save_folder, f"agent-{agent}-camera-boxes.png"), im_out)

In [ ]:
# infrastructure view videos
for agent in range(1, 4):
    camera_sensor = "camera-0"
    frames = CDM.get_frames(sensor=camera_sensor, agent=agent)[1:-1]
    boxes = [
        CDM.get_objects(frame, sensor=camera_sensor, agent=agent) for frame in frames
    ]
    raw_imgs = [CDM.get_image(frame, sensor=camera_sensor, agent=agent) for frame in frames]
    filepath = os.path.join(save_folder, f"agent-{agent}-camera-movie.mp4")
    make_movie(raw_imgs=raw_imgs, boxes=boxes, show_in_notebook=False, save=True, name=filepath)
    

    # frame = CDM.get_frames(sensor=camera_sensor, agent=agent)[frame_idx]
    # img = CDM.get_image(frame=frame, sensor=camera_sensor, agent=agent)
    # pc = CDM.get_lidar(frame=frame, sensor=lidar_sensor, agent=agent)
    # objs = CDM.get_objects(frame=frame, sensor=lidar_sensor, agent=agent)

    # show_lidar_bev_with_boxes(pc=pc, boxes=objs)
    # show_image_with_boxes(img=img, boxes=[], inline=True, return_image=False)
    # show_image_with_boxes(img=img, boxes=objs, inline=True, return_image=False)